In [9]:
import pandas as pd
import numpy as np

df= pd.read_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx")

In [10]:
import pandas as pd
import os

# Define the specific path you provided
file_path = r'C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx'

# Check if file exists before trying to load
if os.path.exists(file_path):
    # Load the 'orders' sheet
    try:
        orders = pd.read_excel(file_path, sheet_name='orders')
        print("✅ Step 1 Success: File loaded.")
        print(f"Dimensions: {len(orders)} rows, {len(orders.columns)} columns")
    except ValueError:
        # Fallback if 'orders' sheet doesn't exist
        print("⚠️ 'orders' sheet not found. Loading first sheet...")
        orders = pd.read_excel(file_path)
        print("✅ Step 1 Success: First sheet loaded.")
else:
    print(f"❌ Error: File not found at {file_path}")
    orders = pd.DataFrame() # Create empty dataframe to prevent crashes in next steps

✅ Step 1 Success: File loaded.
Dimensions: 21864 rows, 14 columns


In [13]:
if not orders.empty:
    print("--- Step 2: Checking for Extra Columns ---")
    extra_cols = [col for col in orders.columns if 'Unnamed' in col]

    if extra_cols:
        print(f"⚠️ Issue Found: {len(extra_cols)} extra columns detected.")
        print(f"   Columns: {extra_cols}")
    else:
        print("✅ Status: Column structure is clean.")
else:
    print("⚠️ Skip: Data not loaded.")

--- Step 2: Checking for Extra Columns ---
⚠️ Issue Found: 2 extra columns detected.
   Columns: ['Unnamed: 12', 'Unnamed: 13']


In [15]:
if not orders.empty:
    print("--- Step 3: Current Data Types ---")
    print(orders.dtypes)
else:
    print("⚠️ Skip: Data not loaded.")

--- Step 3: Current Data Types ---
USER_ID                            object
ORDER_ID                           object
PURCHASE_TS                        object
SHIP_TS                    datetime64[ns]
REFUND_TS                  datetime64[ns]
PRODUCT_NAME                       object
PRODUCT_ID                         object
USD_PRICE                         float64
PURCHASE_PLATFORM                  object
MARKETING_CHANNEL                  object
ACCOUNT_CREATION_METHOD            object
COUNTRY_CODE                       object
Unnamed: 12                       float64
Unnamed: 13                       float64
dtype: object


In [16]:
if not orders.empty:
    print("--- Step 4: Checking Logic Errors ---")
    
    # Create a temporary copy to test logic safely
    check_df = orders.copy()

    # Temporarily convert columns to datetime/numeric for testing
    date_cols = ['PURCHASE_TS', 'SHIP_TS', 'REFUND_TS']
    for col in date_cols:
        if col in check_df.columns:
            check_df[col] = pd.to_datetime(check_df[col], errors='coerce')
    
    if 'USD_PRICE' in check_df.columns:
        check_df['USD_PRICE'] = pd.to_numeric(check_df['USD_PRICE'], errors='coerce')

        # Check A: Ship Date < Purchase Date
        if 'SHIP_TS' in check_df.columns and 'PURCHASE_TS' in check_df.columns:
            impossible = check_df[check_df['SHIP_TS'] < check_df['PURCHASE_TS']]
            if not impossible.empty:
                print(f"⚠️ Logic Error: {len(impossible)} orders have Ship Date < Purchase Date.")
            else:
                print("✅ Date Logic: Valid.")

        # Check B: Prices <= 0
        invalid_prices = check_df[check_df['USD_PRICE'] <= 0]
        if not invalid_prices.empty:
            print(f"⚠️ Logic Error: {len(invalid_prices)} orders have invalid prices ($0 or less).")
        else:
            print("✅ Price Logic: Valid.")
else:
    print("⚠️ Skip: Data not loaded.")

--- Step 4: Checking Logic Errors ---
⚠️ Logic Error: 1997 orders have Ship Date < Purchase Date.
⚠️ Logic Error: 29 orders have invalid prices ($0 or less).


In [17]:
if not orders.empty:
    print("--- Step 5: Quality Check ---")
    
    # Check Duplicates
    if 'ORDER_ID' in orders.columns:
        dup_ids = orders['ORDER_ID'].duplicated().sum()
        if dup_ids > 0:
            print(f"⚠️ Issue: Found {dup_ids} duplicate Order IDs.")
        else:
            print("✅ Duplicates: None found.")
            
    # Check Missing Values
    missing = orders.isnull().sum()
    missing_only = missing[missing > 0]
    
    if not missing_only.empty:
        print("\n⚠️ Missing Data Detected:")
        print(missing_only)
    else:
        print("✅ Missing Data: None.")
else:
    print("⚠️ Skip: Data not loaded.")

--- Step 5: Quality Check ---
⚠️ Issue: Found 145 duplicate Order IDs.

⚠️ Missing Data Detected:
REFUND_TS                  18377
USD_PRICE                      5
MARKETING_CHANNEL             83
ACCOUNT_CREATION_METHOD       83
COUNTRY_CODE                  38
Unnamed: 12                21864
Unnamed: 13                21864
dtype: int64


In [18]:
# Standardize column names
orders.columns = orders.columns.str.lower().str.strip().str.replace(' ', '_')

# Verify the changes
print("\nUpdated Columns:")
print(orders.columns.tolist())

# Save back to the same Excel file (overwrite)
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)



Updated Columns:
['user_id', 'order_id', 'purchase_ts', 'ship_ts', 'refund_ts', 'product_name', 'product_id', 'usd_price', 'purchase_platform', 'marketing_channel', 'account_creation_method', 'country_code', 'unnamed:_12', 'unnamed:_13']


In [19]:
# 1. Fix Dates (The "Wrong" columns)
date_cols = ['purchase_ts', 'ship_ts', 'refund_ts']
for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# 2. Optimize Categories (Optional but Recommended)
cat_cols = ['purchase_platform', 'marketing_channel', 'account_creation_method']
for col in cat_cols:
    orders[col] = orders[col].astype('category')

# 3. Verify the Fix
print("\nUpdated Data Types:")
print(orders.dtypes)

# 4. Save back to the same Excel file (overwrite)
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)



Updated Data Types:
user_id                            object
order_id                           object
purchase_ts                datetime64[ns]
ship_ts                    datetime64[ns]
refund_ts                  datetime64[ns]
product_name                       object
product_id                         object
usd_price                         float64
purchase_platform                category
marketing_channel                category
account_creation_method          category
country_code                       object
unnamed:_12                       float64
unnamed:_13                       float64
dtype: object


In [23]:
# Drop Unnamed columns only if they exist
unnamed_cols = ['Unnamed: 12', 'Unnamed: 13']
existing_cols = [col for col in unnamed_cols if col in orders.columns]

if existing_cols:
    orders.drop(columns=existing_cols, inplace=True)

# Save back to the same Excel file (overwrite)
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)


In [24]:
import pandas as pd

# ==========================================
# SPLIT & SAVE: REVIEW DATA + CLEAN MAIN DATA
# ==========================================

# 1. Define File Paths
main_file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"
review_file_path = r"C:\Users\reddy\Downloads\stockholder_review_data.xlsx"

# 2. Load the Data
try:
    df = pd.read_excel(main_file_path)
    print(f"Original Row Count: {len(df)}")

    # 3. Identify Price Column
    if 'USD_PRICE' in df.columns:
        price_col = 'USD_PRICE'
    elif 'usd_price' in df.columns:
        price_col = 'usd_price'
    else:
        raise ValueError("Could not find Price column.")

    # 4. Create filter for invalid or missing prices
    invalid_mask = (df[price_col].isna()) | (df[price_col] <= 0)

    # 5. Split the Data
    error_data = df[invalid_mask].copy()   # Data for Stakeholders (invalid + missing)
    clean_data = df[~invalid_mask].copy()  # Data for Analysis (valid only)

    # 6. Save the Files
    # Requirement A: Save error data for review
    error_data.to_excel(review_file_path, index=False)
    print(f"✅ SAVED {len(error_data)} invalid/missing rows to: {review_file_path}")

    # Requirement B: Overwrite main file with clean data
    clean_data.to_excel(main_file_path, index=False)
    print(f"✅ UPDATED Main File with {len(clean_data)} clean rows at: {main_file_path}")

except Exception as e:
    print(f"❌ Error: {e}")


Original Row Count: 21864
✅ SAVED 34 invalid/missing rows to: C:\Users\reddy\Downloads\stockholder_review_data.xlsx
✅ UPDATED Main File with 21830 clean rows at: C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx


In [26]:
# Replace missing values in marketing_channel with "Unknown"
if 'marketing_channel' in orders.columns:
    orders['marketing_channel'] = orders['marketing_channel'].fillna('Unknown')

# Verify the fix
print("Missing values after fix:", orders['marketing_channel'].isna().sum())
print(orders['marketing_channel'].value_counts())

# Save back to the same Excel file (overwrite)
orders.to_excel(file_path, index=False)
print(f"✅ Updated file saved at: {file_path}")


Missing values after fix: 0
marketing_channel
direct          17412
email            3251
affiliate         715
social media      322
Unknown            83
unknown            47
Name: count, dtype: int64
✅ Updated file saved at: C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx


In [27]:
# Standardize 'Unknown' entries in marketing_channel column
if 'marketing_channel' in orders.columns:
    orders['marketing_channel'] = orders['marketing_channel'].replace({'unknown': 'Unknown'})


In [28]:
print(orders['marketing_channel'].value_counts())


marketing_channel
direct          17412
email            3251
affiliate         715
social media      322
Unknown           130
Name: count, dtype: int64


In [29]:
# ✅ Drop duplicate Order IDs, keeping the first occurrence
if 'order_id' in orders.columns:
    orders = orders.drop_duplicates(subset='order_id', keep='first')

# Optional: Verify how many rows remain
print(f"✅ Cleaned row count: {len(orders)}")
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)


✅ Cleaned row count: 21685


In [30]:

# Assume orders DataFrame is already loaded
# Ensure datetime conversion
orders['purchase_ts'] = pd.to_datetime(orders['purchase_ts'], errors='coerce')
orders['ship_ts'] = pd.to_datetime(orders['ship_ts'], errors='coerce')

# ✅ Step 1: Calculate shipping delay (in days) for valid rows
valid_mask = orders['ship_ts'] >= orders['purchase_ts']
valid_delays = (orders.loc[valid_mask, 'ship_ts'] - orders.loc[valid_mask, 'purchase_ts']).dt.days

# ✅ Step 2: Compute median delay
median_delay = valid_delays.median()
print(f"Median shipping delay: {median_delay} days")

# ✅ Step 3: Fix invalid rows (ship_ts < purchase_ts)
invalid_mask = orders['ship_ts'] < orders['purchase_ts']
orders.loc[invalid_mask, 'ship_ts'] = orders.loc[invalid_mask, 'purchase_ts'] + pd.to_timedelta(median_delay, unit='D')

# ✅ Step 4: Verify fix
print("Remaining logic errors:", (orders['ship_ts'] < orders['purchase_ts']).sum())

# ✅ Step 5: Save back to Excel
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)
print("✅ Updated file saved with corrected ship dates.")


C:\Users\reddy\AppData\Local\Temp\ipykernel_5768\3525237821.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['purchase_ts'] = pd.to_datetime(orders['purchase_ts'], errors='coerce')
C:\Users\reddy\AppData\Local\Temp\ipykernel_5768\3525237821.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['ship_ts'] = pd.to_datetime(orders['ship_ts'], errors='coerce')


Median shipping delay: 2.0 days
Remaining logic errors: 0
✅ Updated file saved with corrected ship dates.


In [31]:
# ✅ Verify that no orders have Ship Date earlier than Purchase Date
invalid_count = (orders['ship_ts'] < orders['purchase_ts']).sum()

if invalid_count == 0:
    print("✅ Verification Passed: No orders have Ship Date earlier than Purchase Date.")
else:
    print(f"⚠️ Verification Failed: {invalid_count} orders still have Ship Date < Purchase Date.")


✅ Verification Passed: No orders have Ship Date earlier than Purchase Date.


In [32]:
orders['account_creation_method'] = orders['account_creation_method'].fillna('Unknown')
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)
print("✅ Updated file saved with corrected ship dates.")

C:\Users\reddy\AppData\Local\Temp\ipykernel_5768\2617751766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['account_creation_method'] = orders['account_creation_method'].fillna('Unknown')


✅ Updated file saved with corrected ship dates.


In [33]:
# ✅ Verify replacement of missing values in account_creation_method
missing_count = orders['account_creation_method'].isna().sum()
unknown_count = (orders['account_creation_method'] == 'Unknown').sum()

print(f"Missing values after fix: {missing_count}")
print(f"Rows labeled as 'Unknown': {unknown_count}")
print(orders['account_creation_method'].value_counts())


Missing values after fix: 0
Rows labeled as 'Unknown': 83
account_creation_method
desktop    16309
mobile      4220
unknown      729
tablet       319
Unknown       83
tv            25
Name: count, dtype: int64


In [34]:
# ✅ Replace missing values in country_code with "Unknown"
orders['country_code'] = orders['country_code'].fillna('Unknown')

# ✅ Save the changes back to the main file
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)


C:\Users\reddy\AppData\Local\Temp\ipykernel_5768\964557365.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['country_code'] = orders['country_code'].fillna('Unknown')


In [35]:
# ✅ Verify replacement of missing values in country_code
missing_count = orders['country_code'].isna().sum()
unknown_count = (orders['country_code'] == 'Unknown').sum()

print(f"Missing values after fix: {missing_count}")
print(f"Rows labeled as 'Unknown': {unknown_count}")
print(orders['country_code'].value_counts())


Missing values after fix: 0
Rows labeled as 'Unknown': 38
country_code
US    10218
GB     1790
CA      942
AU      887
DE      845
      ...  
RE        1
MZ        1
MH        1
MD        1
LC        1
Name: count, Length: 151, dtype: int64


In [37]:
# ✅ Drop all columns that start with "Unnamed"
unnamed_cols = [col for col in orders.columns if col.strip().startswith("Unnamed")]
orders.drop(columns=unnamed_cols, inplace=True)

# ✅ Save the updated DataFrame to the main file
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)



C:\Users\reddy\AppData\Local\Temp\ipykernel_5768\2281601090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders.drop(columns=unnamed_cols, inplace=True)


In [39]:
# ✅ Display all unique product names 
print(orders['product_name'].unique())

['Nintendo Switch' 'Sony PlayStation 5 Bundle' '27in 4K gaming monitor'
 'JBL Quantum 100 Gaming Headset' 'Dell Gaming Mouse'
 'Acer Nitro V Gaming Laptop' 'Lenovo IdeaPad Gaming 3'
 'Razer Pro Gaming Headset' '27inches 4k gaming monitor']


In [40]:
import pandas as pd

# 1. Load the Data
# Note: Using your local path. If reading the CSV directly, change read_excel to read_csv
file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"

try:
    # Attempt to read as Excel first
    df = pd.read_excel(file_path)
except:
    # Fallback to read as CSV if it's actually a CSV file disguised as xlsx
    try:
        df = pd.read_csv(file_path)
    except:
        # Fallback for sheet name issues
        df = pd.read_excel(file_path, sheet_name=0)

# 2. Standardize Column Names (Best Practice)
# This ensures code works whether columns are 'PRODUCT_NAME' or 'product_name'
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# 3. Standardize Product Name
bad_name = "27inches 4k gaming monitor"
good_name = "27in 4K gaming monitor"

if 'product_name' in df.columns:
    # Check how many exist before change
    count_before = len(df[df['product_name'] == bad_name])
    
    if count_before > 0:
        # Apply the fix
        df.loc[df['product_name'] == bad_name, 'product_name'] = good_name
        
        # 4. Save Changes
        # If it was a CSV, save as CSV. If Excel, save as Excel.
        if file_path.endswith('.csv'):
            df.to_csv(file_path, index=False)
        else:
            df.to_excel(file_path, index=False)
        
        # 5. Verification Report
        print(f"✅ SUCCESS: Merged {count_before} rows of '{bad_name}' into '{good_name}'.")
        print(f"   New total count for '{good_name}': {len(df[df['product_name'] == good_name])}")
    else:
        print(f"ℹ️ No rows found with name: '{bad_name}'. Data might already be clean.")

    # Show current unique products
    print("\nCurrent Unique Products:")
    print(df['product_name'].unique())

else:
    print("❌ Error: Column 'product_name' not found. Check your file headers.")

✅ SUCCESS: Merged 61 rows of '27inches 4k gaming monitor' into '27in 4K gaming monitor'.
   New total count for '27in 4K gaming monitor': 4678

Current Unique Products:
['Nintendo Switch' 'Sony PlayStation 5 Bundle' '27in 4K gaming monitor'
 'JBL Quantum 100 Gaming Headset' 'Dell Gaming Mouse'
 'Acer Nitro V Gaming Laptop' 'Lenovo IdeaPad Gaming 3'
 'Razer Pro Gaming Headset']


In [41]:
import pandas as pd
import numpy as np

class DataQualityAuditor:
    def __init__(self, df):
        self.df = df.copy()
        # Standardize column names for consistent checking
        self.df.columns = self.df.columns.str.lower().str.strip().str.replace(' ', '_')
        
        # Initialize 'issues' column to store error messages
        self.issues = pd.Series([""] * len(self.df), index=self.df.index, dtype='object')
        
        # Initialize 'score' column at 100 (Perfect)
        self.score = pd.Series([100] * len(self.df), index=self.df.index)

    def _log_issue(self, mask, message, penalty=10):
        """
        Core Logic: If a row fails a check (mask=True), 
        append the error message and deduct points.
        """
        if mask.any():
            idx = self.df.index[mask]
            
            # 1. Deduct Score
            self.score.loc[idx] -= penalty
            
            # 2. Append Error Message (Robust string concatenation)
            new_vals = self.issues.loc[idx].apply(lambda x: f"{x}, {message}" if x else message)
            self.issues.loc[idx] = new_vals

    def check_completeness(self, critical_columns):
        """Flag rows with missing values in key columns."""
        for col in critical_columns:
            if col in self.df.columns:
                mask = self.df[col].isnull()
                self._log_issue(mask, f"Missing {col}", penalty=20)

    def check_validity_numeric(self, col, min_val=None, max_val=None):
        """Flag rows with numeric values out of bounds (e.g., Price <= 0)."""
        if col in self.df.columns:
            # Ensure numeric type
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce')
            
            if min_val is not None:
                mask = self.df[col] < min_val
                self._log_issue(mask, f"{col} too low (<{min_val})", penalty=100) # Critical Error
            if max_val is not None:
                mask = self.df[col] > max_val
                self._log_issue(mask, f"{col} too high (>{max_val})", penalty=10)

    def check_logic_dates(self):
        """Check logical order of dates (e.g., Ship Date cannot be before Purchase Date)."""
        date_cols = ['purchase_ts', 'ship_ts', 'refund_ts']
        for col in date_cols:
             if col in self.df.columns:
                self.df[col] = pd.to_datetime(self.df[col], errors='coerce')

        # Rule 1: Ship Date < Purchase Date
        if 'purchase_ts' in self.df.columns and 'ship_ts' in self.df.columns:
            mask = (self.df['ship_ts'].notnull()) & (self.df['purchase_ts'].notnull()) & \
                   (self.df['ship_ts'] < self.df['purchase_ts'])
            self._log_issue(mask, "Ship Date before Purchase", penalty=50)

        # Rule 2: Refund Date < Purchase Date
        if 'purchase_ts' in self.df.columns and 'refund_ts' in self.df.columns:
            mask = (self.df['refund_ts'].notnull()) & (self.df['purchase_ts'].notnull()) & \
                   (self.df['refund_ts'] < self.df['purchase_ts'])
            self._log_issue(mask, "Refund Date before Purchase", penalty=50)
            
    def check_format_regex(self, col, regex):
        """Check text formats using Regex (e.g., Country Code must be 2 uppercase letters)."""
        if col in self.df.columns:
            # Check only non-null values against regex
            mask = self.df[col].notnull() & ~self.df[col].astype(str).str.match(regex)
            self._log_issue(mask, f"Invalid format in {col}", penalty=10)

    def run_audit(self):
        print("--- Starting Data Quality Audit ---")
        
        # 1. Define Critical Checks
        self.check_completeness(['order_id', 'user_id', 'product_name', 'usd_price', 'purchase_ts'])
        self.check_validity_numeric('usd_price', min_val=0.01) # Price must be > 0
        self.check_logic_dates()
        self.check_format_regex('country_code', regex=r'^[A-Z]{2}$') # e.g., 'US', 'IN'
        
        # 2. Compile Results
        self.df['dq_issues'] = self.issues
        self.df['dq_score'] = self.score.clip(lower=0) # Minimum score is 0
        
        # 3. Assign Tier
        conditions = [
            (self.df['dq_score'] == 100),
            (self.df['dq_score'] >= 80),
            (self.df['dq_score'] < 80)
        ]
        choices = ['Perfect', 'Good', 'Critical']
        self.df['dq_tier'] = np.select(conditions, choices, default='Critical')
        
        return self.df

# --- EXECUTION ---
# 1. Load Data
file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"
try:
    df = pd.read_excel(file_path)
except:
    df = pd.read_excel(file_path, sheet_name=0)

# 2. Run Auditor
auditor = DataQualityAuditor(df)
report_df = auditor.run_audit()

# 3. Save Report
report_df.to_csv(r"C:\Users\reddy\Downloads\gamezone_data_quality_report.csv", index=False)

# 4. Display Summary
print("\n--- DATA QUALITY SUMMARY ---")
print(report_df['dq_tier'].value_counts())
print("\n--- SAMPLE ISSUES FOUND ---")
print(report_df[report_df['dq_score'] < 100][['order_id', 'dq_score', 'dq_issues']].head())

--- Starting Data Quality Audit ---

--- DATA QUALITY SUMMARY ---
dq_tier
Perfect    21642
Good          43
Name: count, dtype: int64

--- SAMPLE ISSUES FOUND ---
              order_id  dq_score                       dq_issues
526   06ee8b82fbc46119        90  Invalid format in country_code
671   08feac8f0a020345        90  Invalid format in country_code
1043  0dc92d0562552247        90  Invalid format in country_code
1047  0dda212aaea69940        80             Missing purchase_ts
3581  2fa5682923166358        90  Invalid format in country_code


In [42]:
import pandas as pd

# ==========================================
# MOVE ERRORS TO SEPARATE SHEETS (REVIEW FILE)
# ==========================================

# 1. Define Paths
main_file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"
# Note: Using the path you provided (check for typo 'stackholder' vs 'stockholder')
review_file_path = r"C:\Users\reddy\Downloads\stackholder_review_data.xlsx"

# 2. Load Data
try:
    df = pd.read_excel(main_file_path)
except:
    df = pd.read_excel(main_file_path, sheet_name=0)

# Standardize columns
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# 3. Identify Errors by Category
# A. Invalid Country Codes (Not 2 chars)
country_mask = ~df['country_code'].astype(str).str.match(r'^[A-Z]{2}$')
country_errors = df[country_mask].copy()

# B. Missing Purchase Dates
date_mask = df['purchase_ts'].isnull()
date_errors = df[date_mask].copy()

# C. Invalid Prices (<= 0) - Just in case any remain
price_mask = df['usd_price'] <= 0
price_errors = df[price_mask].copy()

# 4. Identify Clean Data (Remove rows with ANY error)
all_errors_mask = country_mask | date_mask | price_mask
clean_df = df[~all_errors_mask].copy()

# 5. Save to Excel with Multiple Sheets
# We use pd.ExcelWriter to write to multiple sheets in one file
try:
    with pd.ExcelWriter(review_file_path, engine='openpyxl') as writer:
        
        # Write Country Errors
        if not country_errors.empty:
            country_errors.to_excel(writer, sheet_name='Invalid_Country_Code', index=False)
            print(f"📄 Added sheet 'Invalid_Country_Code' with {len(country_errors)} rows.")
        
        # Write Date Errors
        if not date_errors.empty:
            date_errors.to_excel(writer, sheet_name='Missing_Purchase_Date', index=False)
            print(f"📄 Added sheet 'Missing_Purchase_Date' with {len(date_errors)} rows.")
            
        # Write Price Errors
        if not price_errors.empty:
            price_errors.to_excel(writer, sheet_name='Invalid_Prices', index=False)
            print(f"📄 Added sheet 'Invalid_Prices' with {len(price_errors)} rows.")
            
        if country_errors.empty and date_errors.empty and price_errors.empty:
            print("✅ No errors found to move!")
            
    print(f"✅ Review File Saved: {review_file_path}")

    # 6. Update Main File (Keep only Clean Data)
    clean_df.to_excel(main_file_path, index=False)
    print(f"✅ Main File Updated (Clean Data Only): {len(clean_df)} rows.")

except Exception as e:
    print(f"❌ Error saving files: {e}")

📄 Added sheet 'Invalid_Country_Code' with 38 rows.
📄 Added sheet 'Missing_Purchase_Date' with 5 rows.
✅ Review File Saved: C:\Users\reddy\Downloads\stackholder_review_data.xlsx
✅ Main File Updated (Clean Data Only): 21642 rows.


In [43]:
import pandas as pd
import os

# 1. Define Paths (Using the same filenames as before)
main_file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"
review_file_path = r"C:\Users\reddy\Downloads\stackholder_review_data.xlsx"

print("--- STARTING VERIFICATION ---")

# 2. Verify Review File (The "Bad" Data)
if os.path.exists(review_file_path):
    print(f"\n✅ Review File Found: {review_file_path}")
    
    # Load all sheets to check them
    xls = pd.ExcelFile(review_file_path)
    sheet_names = xls.sheet_names
    print(f"   Sheets found: {sheet_names}")
    
    # Check specific sheets
    expected_sheets = ['Invalid_Country_Code', 'Missing_Purchase_Date']
    for sheet in expected_sheets:
        if sheet in sheet_names:
            df_sheet = pd.read_excel(xls, sheet_name=sheet)
            print(f"   - Sheet '{sheet}': Contains {len(df_sheet)} rows.")
        else:
            print(f"   ⚠️ Warning: Sheet '{sheet}' is missing (maybe no errors of this type existed?)")
else:
    print(f"❌ Error: Review file not found at {review_file_path}")

# 3. Verify Main File (The "Clean" Data)
if os.path.exists(main_file_path):
    print(f"\n✅ Main File Found: {main_file_path}")
    try:
        df_main = pd.read_excel(main_file_path)
    except:
        df_main = pd.read_excel(main_file_path, sheet_name=0)
        
    # Standardize columns for checking
    df_main.columns = df_main.columns.str.lower().str.strip().str.replace(' ', '_')
    
    # RE-RUN THE AUDIT LOGIC TO PROVE IT IS CLEAN
    
    # Check 1: Invalid Country Codes
    bad_country = df_main[~df_main['country_code'].astype(str).str.match(r'^[A-Z]{2}$')]
    
    # Check 2: Missing Dates
    bad_dates = df_main[df_main['purchase_ts'].isnull()]
    
    # Check 3: Invalid Prices
    bad_prices = df_main[df_main['usd_price'] <= 0] if 'usd_price' in df_main.columns else []
    
    if len(bad_country) == 0 and len(bad_dates) == 0 and len(bad_prices) == 0:
        print(f"   ✅ SUCCESS: Main file is 100% CLEAN.")
        print(f"      - Invalid Country Codes: 0")
        print(f"      - Missing Dates: 0")
        print(f"      - Invalid Prices: 0")
        print(f"   Total Valid Rows: {len(df_main)}")
    else:
        print(f"   ❌ FAILURE: Main file still has errors!")
        print(f"      - Invalid Country Codes: {len(bad_country)}")
        print(f"      - Missing Dates: {len(bad_dates)}")
        print(f"      - Invalid Prices: {len(bad_prices)}")

else:
    print(f"❌ Error: Main file not found at {main_file_path}")

--- STARTING VERIFICATION ---

✅ Review File Found: C:\Users\reddy\Downloads\stackholder_review_data.xlsx
   Sheets found: ['Invalid_Country_Code', 'Missing_Purchase_Date']
   - Sheet 'Invalid_Country_Code': Contains 38 rows.
   - Sheet 'Missing_Purchase_Date': Contains 5 rows.

✅ Main File Found: C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx
   ✅ SUCCESS: Main file is 100% CLEAN.
      - Invalid Country Codes: 0
      - Missing Dates: 0
      - Invalid Prices: 0
   Total Valid Rows: 21642


In [44]:
import pandas as pd
import os
import sys

class PipelineQualityGate:
    def __init__(self, file_path):
        self.file_path = file_path
        self.df = None
        self.status = True
        self.report = []

    def load_data(self):
        """Loads data and standardizes columns for validation."""
        if not os.path.exists(self.file_path):
            self._log(f"CRITICAL: File not found at {self.file_path}", level="FAIL")
            return False
        
        try:
            try:
                self.df = pd.read_excel(self.file_path)
            except:
                self.df = pd.read_excel(self.file_path, sheet_name=0)
            
            # Standardize columns (Best Practice for Pipelines)
            self.df.columns = self.df.columns.str.lower().str.strip().str.replace(' ', '_')
            return True
        except Exception as e:
            self._log(f"CRITICAL: Failed to read file. Error: {e}", level="FAIL")
            return False

    def _log(self, message, level="INFO"):
        """Internal logger."""
        self.report.append(f"[{level}] {message}")
        if level == "FAIL":
            self.status = False

    def validate_schema(self, required_columns):
        """Ensures all necessary columns exist."""
        missing = [col for col in required_columns if col not in self.df.columns]
        if missing:
            self._log(f"Schema Mismatch! Missing columns: {missing}", level="FAIL")
        else:
            self._log(f"Schema Check Passed: All {len(required_columns)} columns present.", level="PASS")

    def validate_completeness(self, critical_columns):
        """Ensures zero nulls in critical fields."""
        for col in critical_columns:
            if col in self.df.columns:
                null_count = self.df[col].isnull().sum()
                if null_count > 0:
                    self._log(f"Completeness Failure: {col} has {null_count} missing values.", level="FAIL")
                else:
                    self._log(f"Completeness Verified: {col} is 100% populated.", level="PASS")

    def validate_business_logic(self):
        """Checks advanced logic constraints."""
        # 1. Price Integrity
        if 'usd_price' in self.df.columns:
            neg_prices = self.df[self.df['usd_price'] <= 0]
            if not neg_prices.empty:
                self._log(f"Logic Failure: Found {len(neg_prices)} rows with Price <= 0", level="FAIL")
            else:
                self._log("Logic Verified: All prices are positive.", level="PASS")

        # 2. Date Integrity (Ship vs Purchase)
        if 'ship_ts' in self.df.columns and 'purchase_ts' in self.df.columns:
            # Ensure datetime type
            self.df['ship_ts'] = pd.to_datetime(self.df['ship_ts'], errors='coerce')
            self.df['purchase_ts'] = pd.to_datetime(self.df['purchase_ts'], errors='coerce')
            
            bad_dates = self.df[(self.df['ship_ts'] < self.df['purchase_ts'])]
            if not bad_dates.empty:
                self._log(f"Logic Failure: {len(bad_dates)} orders shipped before purchase.", level="FAIL")
            else:
                self._log("Logic Verified: Temporal ordering (Purchase -> Ship) is correct.", level="PASS")

        # 3. Format Integrity (Country Code)
        if 'country_code' in self.df.columns:
            bad_codes = self.df[~self.df['country_code'].astype(str).str.match(r'^[A-Z]{2}$')]
            if not bad_codes.empty:
                self._log(f"Format Failure: {len(bad_codes)} invalid country codes found.", level="FAIL")
            else:
                self._log("Format Verified: All country codes follow ISO 2-char standard.", level="PASS")

    def run_gate(self):
        print("🚀 STARTING PIPELINE QUALITY GATE...")
        print(f"Target File: {self.file_path}\n")
        
        if self.load_data():
            # Define your Pipeline Contract (What MUST be true)
            self.validate_schema(['order_id', 'user_id', 'product_name', 'usd_price', 'purchase_ts', 'country_code'])
            self.validate_completeness(['order_id', 'usd_price', 'purchase_ts', 'country_code'])
            self.validate_business_logic()
        
        print("\n--- GATE RESULTS ---")
        for line in self.report:
            print(line)
        
        print("\n" + "="*30)
        if self.status:
            print("✅ PIPELINE STATUS: GREEN LIGHT (GO)")
            print("   Data is clean, consistent, and ready for ingestion.")
        else:
            print("🛑 PIPELINE STATUS: RED LIGHT (NO GO)")
            print("   Critical errors detected. Do not proceed to ingestion.")
        print("="*30)

# ==========================================
# EXECUTION
# ==========================================
file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"
gate = PipelineQualityGate(file_path)
gate.run_gate()

🚀 STARTING PIPELINE QUALITY GATE...
Target File: C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx


--- GATE RESULTS ---
[PASS] Schema Check Passed: All 6 columns present.
[PASS] Completeness Verified: order_id is 100% populated.
[PASS] Completeness Verified: usd_price is 100% populated.
[PASS] Completeness Verified: purchase_ts is 100% populated.
[PASS] Completeness Verified: country_code is 100% populated.
[PASS] Logic Verified: All prices are positive.
[PASS] Logic Verified: Temporal ordering (Purchase -> Ship) is correct.
[PASS] Format Verified: All country codes follow ISO 2-char standard.

✅ PIPELINE STATUS: GREEN LIGHT (GO)
   Data is clean, consistent, and ready for ingestion.


In [49]:
import pandas as pd
import hashlib

# 1. Load Data
file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"
try:
    df = pd.read_excel(file_path)
except:
    df = pd.read_excel(file_path, sheet_name=0)

# Standardize column names
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# 2. Define the Hashing Function
def generate_sha256(value):
    """
    Converts any value to a string and returns its SHA-256 hash.
    Standardizes data by ignoring types (int vs string).
    """
    if pd.isna(value):
        return None
    # Encode string to bytes, then hash
    return hashlib.sha256(str(value).encode('utf-8')).hexdigest()

print("--- Processing Hashes ---")

# 3. Apply to User ID
if 'user_id' in df.columns:
    df['user_id_sha256'] = df['user_id'].apply(generate_sha256)
    print("✅ Created column: user_id_sha256")

# 4. Apply to Product ID
if 'product_id' in df.columns:
    df['product_id_sha256'] = df['product_id'].apply(generate_sha256)
    print("✅ Created column: product_id_sha256")

# 5. Save the Updated File
df.to_excel(file_path, index=False)
print(f"✅ File saved successfully: {file_path}")

# 6. Preview the Transformation
print("\n--- Sample Output ---")
print(df[['user_id', 'user_id_sha256', 'product_id', 'product_id_sha256']].head())

# ✅ Save the updated DataFrame to the main file
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)

--- Processing Hashes ---
✅ Created column: user_id_sha256
✅ Created column: product_id_sha256
✅ File saved successfully: C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx

--- Sample Output ---
    user_id                                     user_id_sha256 product_id  \
0  2c06175e  683bccf9ecda62f60ff3db0d82c80d1f263ca6cc4a4391...       e682   
1  ee8e5bc2  80fef5010f0a72ae4faf76fd450f129c53c5e0c5bc0352...       e682   
2  9eb4efe0  ff196d75d2d5288c9bb0832015f4de8b2a32377925467e...       8d0d   
3  cac7cbaf  d0b3daf7daa3df1d414f9757f83700cdd0eb872f620d4c...       54ed   
4  6b0230bc  f7fc635c932b51cd9cca47eb9e98eae9ad8b0fa66f89e2...       8d0d   

                                   product_id_sha256  
0  3855c12e3c82068e038acb3c8fafbbbb5f9c5d88613517...  
1  3855c12e3c82068e038acb3c8fafbbbb5f9c5d88613517...  
2  0813cab06a74897cfaa92e87bd20a3771e88ff14239074...  
3  ab46419b315378675d2530b35d6d346d8edeaa30ff9b19...  
4  0813cab06a74897cfaa92e87bd20a3771e88ff14239074...  


In [46]:


# ✅ Save the updated DataFrame to the main file
orders.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)


In [48]:
import pandas as pd
import numpy as np

# Load Data
df = pd.read_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx")

# Standardize columns
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# ---------------------------------------------------------
# SOLUTION 1: Create the 'is_refunded' Flag (Best for Analysis)
# ---------------------------------------------------------
# Logic: If refund_ts has a value -> 1 (True). If it is null -> 0 (False).
df['is_refunded'] = df['refund_ts'].notnull().astype(int)

# ---------------------------------------------------------
# SOLUTION 2: Handling the Date Column Itself (Best for Display)
# ---------------------------------------------------------
# Option A: Keep as NaT (Pandas default) - Best for time-series math
df['refund_ts'] = pd.to_datetime(df['refund_ts'], errors='coerce')

# Option B: Fill with placeholder (Only for exporting to BI tools like Tableau/PowerBI)
# Some tools hate Null dates, so we give them a dummy date far in the future or past.
# df['refund_ts_clean'] = df['refund_ts'].fillna(pd.Timestamp("1900-01-01"))

# ---------------------------------------------------------
# CHECK YOUR METRICS
# ---------------------------------------------------------
refund_rate = df['is_refunded'].mean() * 100
print(f"✅ Data Processed.")
print(f"   - Total Orders: {len(df)}")
print(f"   - Refunded Orders: {df['is_refunded'].sum()}")
print(f"   - Refund Rate: {refund_rate:.1f}%")

# Save the enriched file
df.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)

✅ Data Processed.
   - Total Orders: 21685
   - Refunded Orders: 3444
   - Refund Rate: 15.9%


In [50]:
import pandas as pd
import hashlib

# 1. Load Data
file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"

try:
    df = pd.read_excel(file_path)
except:
    df = pd.read_excel(file_path, sheet_name=0)

# Standardize column names
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# 2. Define the Hashing Function
def generate_sha256(value):
    """
    Converts any value to a string and returns its SHA-256 hash.
    Standardizes data by ignoring types (int vs string).
    """
    if pd.isna(value):
        return None
    return hashlib.sha256(str(value).encode('utf-8')).hexdigest()

print("--- Processing Hashes ---")

# 3. Apply to User ID
if 'user_id' in df.columns:
    df['user_id_sha256'] = df['user_id'].apply(generate_sha256)
    print("✅ Created column: user_id_sha256")

# 4. Apply to Product ID
if 'product_id' in df.columns:
    df['product_id_sha256'] = df['product_id'].apply(generate_sha256)
    print("✅ Created column: product_id_sha256")

# 5. Save the Updated File
df.to_excel(file_path, index=False)
print(f"✅ File saved successfully: {file_path}")

# 6. Preview the Transformation (safe check)
preview_cols = [col for col in ['user_id', 'user_id_sha256', 'product_id', 'product_id_sha256'] if col in df.columns]
print("\n--- Sample Output ---")
print(df[preview_cols].head())


--- Processing Hashes ---
✅ Created column: user_id_sha256
✅ Created column: product_id_sha256
✅ File saved successfully: C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx

--- Sample Output ---
    user_id                                     user_id_sha256 product_id  \
0  2c06175e  683bccf9ecda62f60ff3db0d82c80d1f263ca6cc4a4391...       e682   
1  ee8e5bc2  80fef5010f0a72ae4faf76fd450f129c53c5e0c5bc0352...       e682   
2  9eb4efe0  ff196d75d2d5288c9bb0832015f4de8b2a32377925467e...       8d0d   
3  cac7cbaf  d0b3daf7daa3df1d414f9757f83700cdd0eb872f620d4c...       54ed   
4  6b0230bc  f7fc635c932b51cd9cca47eb9e98eae9ad8b0fa66f89e2...       8d0d   

                                   product_id_sha256  
0  3855c12e3c82068e038acb3c8fafbbbb5f9c5d88613517...  
1  3855c12e3c82068e038acb3c8fafbbbb5f9c5d88613517...  
2  0813cab06a74897cfaa92e87bd20a3771e88ff14239074...  
3  ab46419b315378675d2530b35d6d346d8edeaa30ff9b19...  
4  0813cab06a74897cfaa92e87bd20a3771e88ff14239074...  


In [51]:
import pandas as pd
import numpy as np

# Load Data
df = pd.read_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx")

# Standardize columns
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# ---------------------------------------------------------
# SOLUTION 1: Create the 'is_refunded' Flag (Best for Analysis)
# ---------------------------------------------------------
df['is_refunded'] = df['refund_ts'].notnull().astype(int)

# ---------------------------------------------------------
# SOLUTION 2: Handling the Date Column Itself (Best for Display)
# ---------------------------------------------------------
df['refund_ts'] = pd.to_datetime(df['refund_ts'], errors='coerce')
# Optional: df['refund_ts_clean'] = df['refund_ts'].fillna(pd.Timestamp("1900-01-01"))

# ---------------------------------------------------------
# CHECK YOUR METRICS
# ---------------------------------------------------------
refund_rate = df['is_refunded'].mean() * 100
print(f"✅ Data Processed.")
print(f"   - Total Orders: {len(df)}")
print(f"   - Refunded Orders: {df['is_refunded'].sum()}")
print(f"   - Refund Rate: {refund_rate:.1f}%")

# Save the enriched file (fixed syntax)
df.to_excel(r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx", index=False)


✅ Data Processed.
   - Total Orders: 21685
   - Refunded Orders: 3444
   - Refund Rate: 15.9%


In [1]:
import pandas as pd
import numpy as np
import hashlib
import os

# ==========================================
# MASTER UPDATE SCRIPT
# ==========================================
# 1. Define Paths
main_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"
review_path = r"C:\Users\reddy\Downloads\stockholder_review_data.xlsx"

# 2. Load Data
try:
    df = pd.read_excel(main_path)
except:
    df = pd.read_excel(main_path, sheet_name=0)

# Standardize Columns
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

# 3. APPLY ALL FEATURES (To the ENTIRE dataset)
print("--- Applying Features to All Rows ---")

# A. SHA-256 Hashing
def get_sha256(val):
    if pd.isna(val): return None
    return hashlib.sha256(str(val).encode('utf-8')).hexdigest()

if 'user_id' in df.columns: 
    df['user_id_sha256'] = df['user_id'].apply(get_sha256)
if 'product_id' in df.columns: 
    df['product_id_sha256'] = df['product_id'].apply(get_sha256)

# B. Operational & Flags
if 'ship_ts' in df.columns and 'purchase_ts' in df.columns:
    df['days_to_ship'] = (df['ship_ts'] - df['purchase_ts']).dt.days
    df['is_late_shipment'] = (df['days_to_ship'] > 3).astype(int)

# C. Refund Logic
if 'refund_ts' in df.columns:
    df['is_refunded'] = df['refund_ts'].notnull().astype(int)

# D. Lifecycle (Sorting required)
if 'user_id' in df.columns and 'purchase_ts' in df.columns:
    df = df.sort_values(by=['user_id', 'purchase_ts'])
    df['order_rank'] = df.groupby('user_id').cumcount() + 1
    df['is_returning'] = (df['order_rank'] > 1).astype(int)

# 4. SPLIT DATA (Clean vs Error)
print("--- Splitting Data ---")

# Define Error Logic
mask_country = ~df['country_code'].astype(str).str.match(r'^[A-Z]{2}$')
mask_dates = df['purchase_ts'].isnull()
mask_prices = (df['usd_price'].isnull()) | (df['usd_price'] <= 0)

# Combine for "Clean" Data
total_error_mask = mask_country | mask_dates | mask_prices
clean_df = df[~total_error_mask].copy()

# 5. SAVE UPDATES

# A. Update MAIN FILE (Clean Only)
clean_df.to_excel(main_path, index=False)
print(f"✅ Main File Saved: {len(clean_df)} rows")

# B. Update REVIEW FILE (Errors + New Columns)
# We write multiple sheets for easier review
with pd.ExcelWriter(review_path, engine='openpyxl') as writer:
    if mask_country.any():
        df[mask_country].to_excel(writer, sheet_name='Invalid_Country', index=False)
    if mask_dates.any():
        df[mask_dates].to_excel(writer, sheet_name='Missing_Dates', index=False)
    if mask_prices.any():
        df[mask_prices].to_excel(writer, sheet_name='Invalid_Prices', index=False)

print(f"✅ Stakeholder Review File Saved with new columns.")

--- Applying Features to All Rows ---
--- Splitting Data ---
✅ Main File Saved: 21642 rows
✅ Stakeholder Review File Saved with new columns.


In [3]:
import pandas as pd

# 1. Load the Main File
file_path = r"C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx"
try:
    df = pd.read_excel(file_path)
except:
    df = pd.read_excel(file_path, sheet_name=0)

# 2. Drop the Columns
# We use errors='ignore' so the script doesn't crash if they are already gone
cols_to_drop = ['unnamed:_12', 'unnamed:_13']
df.drop(columns=cols_to_drop, errors='ignore', inplace=True)

# 3. Save Changes
df.to_excel(file_path, index=False)

print(f"✅ Dropped columns {cols_to_drop}")
print(f"✅ File saved: {file_path}")

✅ Dropped columns ['unnamed:_12', 'unnamed:_13']
✅ File saved: C:\Users\reddy\Downloads\gamezone-orders-data (1).xlsx
